<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

## Model Submission Guide: Dates Classification Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/i7c1w0k3/date_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Separate data into X_train, y_train, and X_test
import pandas as pd
y_train = pd.read_csv("date_competition_data/y_train.csv")
y_train_labels = y_train.idxmax(axis=1)

X_train = pd.read_csv("date_competition_data/X_train.csv")
X_test=pd.read_csv("date_competition_data/X_test.csv")

X_train.head()

,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
0,2385.5110,849.6287,548.4377,0.7638,681.0944,0.9874,368969,0.7596,1.5492,0.8045,...,0.3835,2.5413,2.7010,4.7956,-4.570224e+10,-31309285376,-31791349760,54.5422,46.1706,47.5687
1,2360.4670,919.1130,479.2314,0.8533,661.2138,0.9657,355591,0.6926,1.9179,0.7744,...,0.8084,13.2536,10.8915,5.0327,-6.173153e+09,-10177111040,-13946608640,20.5354,27.7553,33.2095
2,1059.0460,378.1419,270.6614,0.6983,319.2416,0.9883,80990,0.7521,1.3971,0.8968,...,-0.1599,2.2247,2.4509,2.2165,-9.834114e+09,-9082079232,-8377428480,54.3915,53.0591,50.5000
3,2279.7581,770.8477,654.0494,0.5292,708.2852,0.9945,396190,0.7632,1.1786,0.9527,...,1.6654,5.7796,9.1424,7.8056,-1.841185e+10,-23242752000,-23591233536,34.5177,38.9835,38.4695
4,1843.9160,647.9982,479.8817,0.6720,556.0906,0.9906,245181,0.7818,1.3503,0.8977,...,-0.3694,2.4534,2.2759,2.5079,-4.665024e+10,-40748339200,-32199612416,66.5225,62.6833,56.9092


##2.   Preprocess data using Sklearn / Write and Save Preprocessor function


In [4]:
# Simple Preprocessor with sklearn 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [5]:
# Here is where we actually write the preprocessor function:
def preprocessor(data):
    preprocessed_data=scaler.transform(data)
    return preprocessed_data

In [6]:
# check shape of X data after preprocessing it using our new function
preprocessor(X_train).shape

(718, 33)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(16, input_dim=33, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(7, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), y_train, 
               epochs = 15, validation_split=0.25) 

#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

#This is the unique rest api that powers this class section's Date Classification Playground -- make sure to update the apiurl for new competition deployments
apiurl="https://lqxssftz68.execute-api.us-east-1.amazonaws.com/prod/m"

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1762


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,softmax_act,relu_act,loss,optimizer,memory_size,username,version
0,92.22%,89.94%,90.05%,90.76%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,3
1,84.44%,69.66%,70.97%,75.14%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,1
2,81.11%,74.93%,76.89%,74.75%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,4
3,72.22%,50.56%,56.43%,58.57%,sklearn,False,False,SVC,nan,693.000000,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,2
4,68.89%,47.35%,52.38%,53.24%,keras,False,True,Sequential,4.000000,1591.000000,4.000000,1.000000,3.000000,str,SGD,7416.000000,Ind_Eng_Op_Rsrch,5


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model_2 = Sequential()
model_2.add(Dense(128, input_dim=33, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))

model_2.add(Dense(7, activation='softmax')) 
                                            
# Compile model
model_2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), y_train, 
               epochs = 10, validation_split=0.25) 

Epoch 1/10
17/17 [==============================] - 2s 37ms/step - loss: 1.9008 - accuracy: 0.1654 - val_loss: 1.7296 - val_accuracy: 0.3500
Epoch 2/10
17/17 [==============================] - 0s 6ms/step - loss: 1.7244 - accuracy: 0.3309 - val_loss: 1.5625 - val_accuracy: 0.5278
Epoch 3/10
17/17 [==============================] - 0s 7ms/step - loss: 1.5808 - accuracy: 0.4275 - val_loss: 1.4263 - val_accuracy: 0.6056
Epoch 4/10
17/17 [==============================] - 0s 11ms/step - loss: 1.4557 - accuracy: 0.5056 - val_loss: 1.3168 - val_accuracy: 0.6222
Epoch 5/10
17/17 [==============================] - 0s 9ms/step - loss: 1.3702 - accuracy: 0.5576 - val_loss: 1.2254 - val_accuracy: 0.6389
Epoch 6/10
17/17 [==============================] - 0s 9ms/step - loss: 1.3115 - accuracy: 0.5706 - val_loss: 1.1494 - val_accuracy: 0.6556
Epoch 7/10
17/17 [==============================] - 0s 7ms/step - loss: 1.2080 - accuracy: 0.6264 - val_loss: 1.0756 - val_accuracy: 0.6722
Epoch 8/10
17/17 [

In [15]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [16]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model_2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 6

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1762


In [17]:
# Compare two or more models
data=mycompetition.compare_models([5,6], verbose=1)
mycompetition.stylize_compare(data)

,Model_5_Layer,Model_5_Shape,Model_5_Params,Model_6_Layer,Model_6_Shape,Model_6_Params
0,Dense,"[None, 16]",544.000000,Dense,"[None, 128]",4352
1,Dense,"[None, 16]",272.000000,Dropout,"[None, 128]",0
2,Dense,"[None, 32]",544.000000,Dense,"[None, 64]",8256
3,Dense,"[None, 7]",231.000000,Dense,"[None, 64]",4160
4,None,None,nan,Dropout,"[None, 64]",0
5,None,None,nan,Dense,"[None, 64]",4160
6,None,None,nan,Dense,"[None, 7]",455


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt


#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=33, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(7, activation='softmax')) 
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 10s]
val_accuracy: 0.6666666666666666

Best val_accuracy So Far: 0.6666666666666666
Total elapsed time: 00h 00m 27s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=5)

Epoch 1/5
23/23 [==============================] - 2s 5ms/step - loss: 2.0253 - accuracy: 0.5655
Epoch 2/5
23/23 [==============================] - 0s 5ms/step - loss: 1.2839 - accuracy: 0.6741
Epoch 3/5
23/23 [==============================] - 0s 4ms/step - loss: 0.8809 - accuracy: 0.7214
Epoch 4/5
23/23 [==============================] - 0s 3ms/step - loss: 0.6308 - accuracy: 0.7354
Epoch 5/5
23/23 [==============================] - 0s 3ms/step - loss: 0.4785 - accuracy: 0.7535


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1762


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,dropout_layers,softmax_act,relu_act,loss,optimizer,memory_size,username,version
0,92.22%,89.94%,90.05%,90.76%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,3
1,84.44%,69.66%,70.97%,75.14%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,1
2,81.11%,74.93%,76.89%,74.75%,sklearn,False,False,GradientBoostingClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,4
3,76.67%,57.64%,56.93%,64.29%,keras,False,True,Sequential,4.000000,38599.000000,4.000000,nan,1.000000,3.000000,str,RMSprop,155448.000000,Ind_Eng_Op_Rsrch,7
4,72.22%,50.56%,56.43%,58.57%,sklearn,False,False,SVC,nan,693.000000,nan,nan,nan,nan,nan,nan,nan,Ind_Eng_Op_Rsrch,2
5,68.89%,47.35%,52.38%,53.24%,keras,False,True,Sequential,4.000000,1591.000000,4.000000,nan,1.000000,3.000000,str,SGD,7416.000000,Ind_Eng_Op_Rsrch,5
6,68.89%,43.27%,37.64%,51.43%,keras,False,True,Sequential,7.000000,21383.000000,5.000000,2.000000,1.000000,4.000000,str,SGD,86872.000000,Ind_Eng_Op_Rsrch,6


In [25]:
# Compare two or more models
data=mycompetition.compare_models([5, 6, 7], verbose=1)
mycompetition.stylize_compare(data)

,Model_5_Layer,Model_5_Shape,Model_5_Params,Model_6_Layer,Model_6_Shape,Model_6_Params,Model_7_Layer,Model_7_Shape,Model_7_Params
0,Dense,"[None, 16]",544.000000,Dense,"[None, 128]",4352,Dense,"[None, 64]",2176.000000
1,Dense,"[None, 16]",272.000000,Dropout,"[None, 128]",0,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 32]",544.000000,Dense,"[None, 64]",8256,Dense,"[None, 448]",29120.000000
3,Dense,"[None, 7]",231.000000,Dense,"[None, 64]",4160,Dense,"[None, 7]",3143.000000
4,None,None,nan,Dropout,"[None, 64]",0,None,None,nan
5,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
6,None,None,nan,Dense,"[None, 7]",455,None,None,nan
